In [ ]:
%reload_ext autoreload
%autoreload 2

from sarpu.data_processing import *
from sarpu.paths_and_names import *

import numpy as np
import pandas as pd
import requests

import sklearn.model_selection

# Input Data

In [ ]:
# Names and locations
data_folder= "../../Data/"
data_name = "mushroom"
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data"

In [ ]:
# Creation information
nb_splits = 5
test_size = 0.2

# Prepare folders

In [ ]:
# Prepare folders
data_folder_original = original_data_path(data_folder,data_name)
!mkdir -p $data_folder_original
data_folder_processed = processed_data_path(data_folder,data_name)
!mkdir -p $data_folder_processed
data_folder_partitions = partitions_data_path(data_folder,data_name)
!mkdir -p $data_folder_partitions

# Download

In [ ]:
#download and unzip data

unprocessed_data_path = os.path.join(data_folder_original,url.split("/")[-1])

if not os.path.exists(unprocessed_data_path):
    r = requests.get(url, allow_redirects=True)
    open(unprocessed_data_path, 'wb').write(r.content)

In [ ]:
#read data to pandas dataframe



# 7. Attribute Information: (classes: edible=e, poisonous=p)
#      1. cap-shape:                bell=b,conical=c,convex=x,flat=f,
#                                   knobbed=k,sunken=s
#      2. cap-surface:              fibrous=f,grooves=g,scaly=y,smooth=s
#      3. cap-color:                brown=n,buff=b,cinnamon=c,gray=g,green=r,
#                                   pink=p,purple=u,red=e,white=w,yellow=y
#      4. bruises?:                 bruises=t,no=f
#      5. odor:                     almond=a,anise=l,creosote=c,fishy=y,foul=f,
#                                   musty=m,none=n,pungent=p,spicy=s
#      6. gill-attachment:          attached=a,descending=d,free=f,notched=n
#      7. gill-spacing:             close=c,crowded=w,distant=d
#      8. gill-size:                broad=b,narrow=n
#      9. gill-color:               black=k,brown=n,buff=b,chocolate=h,gray=g,
#                                   green=r,orange=o,pink=p,purple=u,red=e,
#                                   white=w,yellow=y
#     10. stalk-shape:              enlarging=e,tapering=t
#     11. stalk-root:               bulbous=b,club=c,cup=u,equal=e,
#                                   rhizomorphs=z,rooted=r,missing=?
#     12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
#     13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
#     14. stalk-color-above-ring:   brown=n,buff=b,cinnamon=c,gray=g,orange=o,
#                                   pink=p,red=e,white=w,yellow=y
#     15. stalk-color-below-ring:   brown=n,buff=b,cinnamon=c,gray=g,orange=o,
#                                   pink=p,red=e,white=w,yellow=y
#     16. veil-type:                partial=p,universal=u
#     17. veil-color:               brown=n,orange=o,white=w,yellow=y
#     18. ring-number:              none=n,one=o,two=t
#     19. ring-type:                cobwebby=c,evanescent=e,flaring=f,large=l,
#                                   none=n,pendant=p,sheathing=s,zone=z
#     20. spore-print-color:        black=k,brown=n,buff=b,chocolate=h,green=r,
#                                   orange=o,purple=u,white=w,yellow=y
#     21. population:               abundant=a,clustered=c,numerous=n,
#                                   scattered=s,several=v,solitary=y
#     22. habitat:                  grasses=g,leaves=l,meadows=m,paths=p,
#                                   urban=u,waste=w,woods=d

header = [
    "class",
    "cap-shape",
    "cap-surface",
    "cap-color",
    "bruises",
    "odor",
    "gill-attachment",
    "gill-spacing",
    "gill-size",
    "gill-color",
    "stalk-shape",
    "stalk-root",
    "stalk-surface-above-ring",
    "stalk-surface-below-ring",
    "stalk-color-above-ring",
    "stalk-color-below-ring",
    "veil-type",
    "veil-color",
    "ring-number",
    "ring-type",
    "spore-print-color",
    "population",
    "habitat"
]

multival=header[1:]

df = pd.read_csv(unprocessed_data_path, names=header).dropna()
df



In [ ]:
#class distribution

df["class"].value_counts()


In [ ]:
# Make poisenous positive class

df.loc[df["class"]=="e","class"]=0
df.loc[df["class"]=="p","class"]=1

# df

In [ ]:
#binarize multivalued features

for column in multival:
    values = list(set(df[column]))
    if len(values)>2:
        df = binarize(df, column)
    elif len(values)==2:
        df.loc[df[column]==values[0],column]=-1
        df.loc[df[column]==values[1],column]=1
    else: # drop useless features
        print(column, values)
        df=df.drop(column, axis=1)

# df

In [ ]:
#normalize
for column in df.columns.values:
    df[column]=pd.to_numeric(df[column])

normalized_df=(df.astype(float)-df.min())/(df.max()-df.min())*2-1
normalized_df["class"] = df["class"]
df = normalized_df
# df

In [ ]:
#move class to back

cols = list(df.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('class')) #Remove class from list
df = df[cols+['class']]

df

In [ ]:
# make numpy array

import numpy as np

xy = df.values

x = xy[:,:-1].astype(float)
y = xy[:,-1].astype(int)

x_pos = x[y==1]
x_neg = x[y==0]


In [ ]:
#Save data and true classes
np.savetxt(data_path(data_folder, data_name), x)
np.savetxt(classlabels_path(data_folder, data_name), y,fmt='%d')

# Different dataset partitions (train/test and class prior)

In [ ]:
sss = sklearn.model_selection.StratifiedShuffleSplit(n_splits=nb_splits, test_size=test_size, random_state=0)
splits = list(sss.split(x,y))

In [ ]:
#save partitions. 0 means not in data, 1 means in train partition, 2 means in test partition

for i, (train,test) in enumerate(splits):
    partition = np.zeros_like(y,dtype=int)
    partition[train]=1
    
    partition[test]=2    
    np.savetxt(partition_path(data_folder,data_name, i), partition, fmt='%d')
   